## Bài toán Mercari Price Suggestion:
#### **1. Tóm tắt**
#### - Sử dụng bộ dữ liệu đã cho để có thể đoán ra được giá của các sản phẩm.
#### - Mô hình đánh giá sử dụng: RMSLE (Root Mean Squared Logarithmic Error - độ đo được sử dụng trong Kaggle):

\begin{align*}
\epsilon = \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 }
\end{align*}<br>

> - ϵ là giá trị RMSLE (score)
> - n là tổng số quan sát trong tập dữ liệu
> - pi là giá dự đoán của sản phẩm i
> - ai là giá trị thực của sản phẩm i
> - log(x) là logarit cơ số e (Logarit tự nhiên) của x

#### **2. Hướng giải quyết**
Đây là bài toán hồi quy. Dữ liệu vào chỉ thuộc dạng text và lable, nên cần chuyển các đặc trưng về dạng số thực hoặc vector để xử lý tuyến tính.

* Sử dụng tfidf vectorizer để xử lý dữ liệu dạng văn bản
* Sử dụng label binarizer để xử lý dữ liệu dạng lable

### Import



In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import shutil
import datetime
import gc
from tqdm import tqdm

import pandas as pd
import numpy as np
from numpy import median

from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

import re
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# **1. Đọc dữ liệu**

# **1.1. Giải nén dữ liệu**

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z

In [ ]:
!unzip -n /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip -n /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

Các file csv là train.vsv, test_stg2.tsv bao gồm dữ liệu để train, test, và file submit mẫu sample_submission_stg2.csv.

# **1.2. Nhìn qua dữ liệu**

In [ ]:
# Đọc dữ liệu train.tsv, test.tsv với pandas
train = pd.read_csv('/kaggle/working/train.tsv', sep='\t')
test = pd.read_csv('/kaggle/working/test_stg2.tsv', sep='\t')

In [ ]:
train.info()

In [ ]:
test.info()

In [ ]:
train.isnull().any()

In [ ]:
train.head(5)

In [ ]:
test.head(5)

# **1.3. Nhận xét về dữ liệu**

Tập train bao gồm 8 trường, trong đó:
* Có ba trường là thuộc tính int64: train_id, item_condition_id, shipping
* Một trường là thuộc tính float64: price
* Ba trường là thuộc tính object: name, categrory_name, brand_name

Tập test cũng gồm các trường giống như tập train loại trừ chứa trường price, là trường cần phải đoán

---
* Tập dữ liệu có hai trường là category là item_condition_id và shipping
* Trường brand name là text nhưng mỗi sản phẩm chỉ có một brand nên có thể coi là category để xử lý 
* Trường category_name là text nhưng mỗi category được ngăn cách nhau bởi dấu "/" nên có thể tách ra thành các từ hoặc cụm từ theo dấu "/" và coi mỗi phần tử được tách ra đó là môt category hoặc xử dụng trực tiếp các phương pháp text để xử lý.
* Trường name và item_description là thuần text
* Các trường category_name, brand_name, item_description có giá trị null nên cần sử dụng các phương pháp fillnull để xử lý dữ liệu
* Trường price có thể có những giá trị nhỏ hơn hoặc bằng 0, cần phải xử lý những giá trị này
* Trường price có thể log lên để tránh khi tính toán kết quả ra giá trị quá bé và bị làm tròn thành không

# **1.4. Đọc và clean dữ liệu**

#### Đọc dữ liệu và đồng thời tách category_name ra thành các từ hoặc cụm từ theo dấu "/" và coi mỗi phần tử được tách ra là một sub-category

In [ ]:
# Split category_name by '/' into subcategories
def split_cat(text):
    try:
        return text.split("/")
    except:
        return ("No Label", "No Label", "No Label")

In [ ]:
# Read input and split category_name by '/' into subcategories
train = pd.read_csv('/kaggle/working/train.tsv', sep='\t', 
                      dtype={'item_condition_id': 'category', 'shipping': 'category'}, 
                      converters={'category_name': split_cat})
test = pd.read_csv('/kaggle/working/test_stg2.tsv', sep='\t', 
                     dtype={'item_condition_id': 'category', 'shipping': 'category'}, 
                     converters={'category_name': split_cat})

In [ ]:
print('Shape of train data: ', train.shape)
print('Shape of test data: ', test.shape)

- Dữ liệu *train* bao gồm 1482535 hàng và 8 cột
- Dữ liệu *test* bao gồm 3460725 hàng và 7 cột

#### Xử lý missing data

* Như đã phân tích ở trên: category_name, brand_name và item_description là những trường có giá trị null
* Có thể thay các giá trị null này bằng một dữ liệu dạng text khác như 'missing' và coi đó là một brand_name

#### Chia category_name thành các sub-category và thay các giá trị null bằng text 'missing'

In [ ]:
# general category
train['gencat_name'] = train['category_name'].str.get(0).fillna('missing').astype('category')
# sub-category 1
train['subcat1_name'] = train['category_name'].str.get(1).fillna('missing').astype('category')
# sub-category 2
train['subcat2_name'] = train['category_name'].str.get(2).fillna('missing').astype('category')
train.drop('category_name', axis=1, inplace=True)

In [ ]:
# general category
test['gencat_name'] = test['category_name'].str.get(0).fillna('missing').astype('category')
# sub-category 1
test['subcat1_name'] = test['category_name'].str.get(1).fillna('missing').astype('category')
# sub-category 2
test['subcat2_name'] = test['category_name'].str.get(2).fillna('missing').astype('category')
test.drop('category_name', axis=1, inplace=True)

#### Thay các giá trị null trong trường brand_name và item_description bằng text 'missing'

In [ ]:
# Replace null values with 'mising'
train['item_description'].fillna('missing', inplace=True)
train['brand_name'] = train['brand_name'].fillna('missing').astype('category')

test['item_description'].fillna('missing', inplace=True)
test['brand_name'] = test['brand_name'].fillna('missing').astype('category')

#### Xử lý hàng trùng lặp

In [ ]:
train[train.duplicated()]

* Không có hàng nào trùng lặp với nhau

#### Kiểm tra liệu dữ liệu còn có null hay không

In [ ]:
train.isnull().any()

* Không còn null trong dữ liệu

### Loại bỏ những hàng có giá không hợp lệ (price <= 0)

In [ ]:
# remove row with price <= 0
print('Removed {} rows' .format(len(train[train.price<=0])))
train = train[train.price > 0].reset_index(drop=True)

# **2. Tổng quan dữ liệu**
### In ra phân phối giá trị của dữ liệu để biết khoảng dữ liệu tập trung, khoảng dữ liệu còn rời rạc, thiếu sót để có cách giải quyết.

#### Trường name

In [ ]:
train.name.describe()

#### item_condition_id

In [ ]:
train.item_condition_id.describe()

In [ ]:
condition_count = Counter(list(train.item_condition_id))
x, y = zip(*condition_count.most_common())
plt.figure(figsize=[8,6])
plt.bar(x, y, )
for i, val in enumerate(y):
           plt.annotate(val, (x[i], y[i]), color='b')
plt.xlabel('item condition')
plt.ylabel('count')
plt.grid(False, axis='x')
plt.show()

##### Phần lớn các sản phẩm nằm trong condition 1

#### brand_name

In [ ]:
train.brand_name.describe()

- Có 4808 brand_name khác nhau
- Trong đó brand_name 'missing' chiếm số lượng nhiều nhất (632336 điểm trên tổng số 1481661 điểm)

In [ ]:
brand_count = Counter(list(train.brand_name.values))
x, y = zip(*brand_count.most_common(15))

plt.figure(figsize=[6,5])
plt.barh(x, y)
for i, val in enumerate(y):
           plt.annotate(val, (y[i], x[i]), color='b')
plt.gca().invert_yaxis()
plt.ylabel('Brand name')
plt.xlabel('count')
plt.grid(False, axis='y')
plt.show()

In [ ]:
brand_missing = train[train.brand_name=='missing'].shape[0]
if brand_missing == 0:
    print('Không có giá trị missing trong trường brand_name.')
else:
    print('brand_name missing có {} điểm, chiếm {:.2f} % của data.' .format(brand_missing, 100.0*brand_missing/train.shape[0]))

#### gencat_name

In [ ]:
train.gencat_name.describe()

- Có 11 loại gencat_name khác nhau
- Trong đó gencat_name 'Women' chiếm số lượng nhiều nhất (663990 điểm trên tổng số 1481661 điểm)

In [ ]:
# Show top 10 most common general category count
gencat_count = Counter(list(train.gencat_name.values))
x, y = zip(*gencat_count.most_common(15))
plt.figure(figsize=[6,5])
plt.barh(x, y)
for i, val in enumerate(y):
           plt.annotate(val, (y[i], x[i]), color='b')
plt.gca().invert_yaxis()
plt.ylabel('General category')
plt.xlabel('count')
plt.grid(False, axis='y')
plt.show()

In [ ]:
gencat_missing = train[train.gencat_name=='missing'].shape[0]
if gencat_missing == 0:
    print('Không có giá trị missing trong trường gencat_name.')
else:
    print('gencat_name missing có {} điểm, chiếm {:.2f} % của data.' .format(gencat_missing, 100.0*gencat_missing/train.shape[0]))

#### subcat1_name

In [ ]:
train.subcat1_name.describe()

- Có 114 loại subcat1_name khác nhau
- Trong đó subcat1_name 'Athletic Apparel' chiếm số lượng nhiều nhất (134321 điểm trên tổng số 1481661 điểm)

In [ ]:
# Show top 10 most common sub-category1 count
subcat1_count = Counter(list(train.subcat1_name.values))
x, y = zip(*subcat1_count.most_common(10))
plt.figure(figsize=[6,5])
plt.barh(x, y)
for i, val in enumerate(y):
           plt.annotate(val, (y[i], x[i]), color='b')
plt.gca().invert_yaxis()
plt.ylabel('Sub-category1')
plt.xlabel('count')
plt.grid(False, axis='y')
plt.show()

In [ ]:
subcat1_missing = train[train.subcat1_name=='missing'].shape[0]
if subcat1_missing == 0:
    print('Không có giá trị missing trong trường subcat1_name.')
else:
    print('subcat1_name missing có {} điểm, chiếm {:.2f} % của data.' .format(subcat1_missing, 100.0*subcat1_missing/train.shape[0]))

#### subcat2_name

In [ ]:
train.subcat2_name.describe()

- Có 871 loại subcat2_name khác nhau
- Trong đó subcat2_name 'Pants, Tights, Leggings' chiếm số lượng nhiều nhất (60152 điểm trên tổng số 1481661 điểm)

In [ ]:
# Show top 10 most common sub-category2 count
subcat2_count = Counter(list(train.subcat2_name.values))
x, y = zip(*subcat2_count.most_common(10))
plt.figure(figsize=[6,5])
plt.barh(x, y)
for i, val in enumerate(y):
           plt.annotate(val, (y[i], x[i]), color='b')
plt.gca().invert_yaxis()
plt.ylabel('Sub-category2')
plt.xlabel('count')
plt.grid(False, axis='y')
plt.show()

In [ ]:
subcat2_missing = train[train.subcat2_name=='missing'].shape[0]
if subcat2_missing == 0:
    print('Không có giá trị missing trong trường subcat2_name.')
else:
    print('subcat2_name missing có {} điểm, chiếm {:.2f} % của data.' .format(subcat2_missing, 100.0*subcat2_missing/train.shape[0]))

#### item_description

In [ ]:
item_description_missing = train[train.item_description=='missing'].shape[0]
if item_description_missing == 0:
    print('Không có giá trị missing trong trường item_description.')
else:
    print('item_description missing có {} điểm, chiếm {:.4f} % của data.' .format(item_description_missing, 100.0*item_description_missing/train.shape[0]))

**Nhận xét:** Số điểm missing của item_description không đáng kể, chỉ có 4 điểm so với 1482535 điểm của dữ liệu nên ta có thể bỏ qua và xóa những hàng này đi

##### Xóa các hàng với trường item_description là 'missing'

In [ ]:
# remove item with missing description
print('Removed {} rows' .format(item_description_missing))
train = train[train.item_description != 'missing'].reset_index(drop=True)

##### Kiểm tra sau khi xóa

In [ ]:
item_description_missing = train[train.item_description=='missing'].shape[0]
if item_description_missing == 0:
    print('Không có giá trị missing trong trường item_description.')
else:
    print('item_description missing có {} điểm, chiếm {:.4f} % của data.' .format(item_description_missing, 100.0*item_description_missing/train.shape[0]))

#### price

In [ ]:
# price distributuon graph
sns.distplot(train['price'], kde=True)

**Nhận xét:** dữ liệu lệch quá nhiều.

**Giải pháp:** Thay vì đánh giá price, ta có thể đánh giá hàm log của price.

**Biểu đồ giá trị log của price:**

In [ ]:
sns.distplot(np.log1p(train['price']), kde=False)

**Nhận xét:** Phân bố của hàm log của price có dạng gần với phân phối chuẩn, nên có thể sử dụng để làm output dự đoán cho bài toán hồi quy.

#### Thay giá trị của price với log(price)

In [ ]:
train['price'] = np.log1p(train['price'])

# **3. Tiền xử lý**
Trong ngôn ngữ tự nhiên, có những từ ngữ, ký hiệu không mang nhiều ý nghĩa trong câu, có thể loại bỏ những từ ngữ, ký hiệu này, làm sạch dữ liệu để việc dự đoán đạt kết quả tốt nhất.

- Loại bỏ các ký tự, từ ngữ không mang nhiều ý nghĩa như dấu câu, line feed trong trường 'name', 'item_description'.

- Loại các từ ngữ không mang nhiều ý nghĩa trong câu như 'your, he, him, who, this, if, while,.... ' (các từ này còn gọi là stopwords) trong trường 'item_description'.

#### Thay thế "ký tự xuống dòng, dấu ngoặc kép, line feed, ký tự giữa các từ" với "space" đồng thời loại bỏ stopword:

In [ ]:
# Pre-process names
def preprocess_name(text_col):
    preprocessed_names = []
    for sentence in tqdm(text_col.values):
        # Replace "carriage return" with "space".
        sent = sentence.replace('\\r', ' ')
        # Replace "quotes" with "space".
        sent = sent.replace('\\"', ' ')
        # Replace "line feed" with "space".
        sent = sent.replace('\\n', ' ')
        # Replace characters between words with "space".
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # to lowercase
        preprocessed_names.append(sent.lower().strip())
    return preprocessed_names

# Pre-process descriptions
stopwords = stopwords.words('english')
def preprocess_desc(text_col):
    preprocessed_descs = []
    for sentence in tqdm(text_col.values):
        # Replace "carriage return" with "space".
        sent = sentence.replace('\\r', ' ')
        # Replace "quotes" with "space".
        sent = sent.replace('\\"', ' ')
        # Replace "line feed" with "space".
        sent = sent.replace('\\n', ' ')
        # Replace characters between words with "space".
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # Removing stopwords.
        sent = ' '.join(e for e in sent.split() if e not in stopwords)
        preprocessed_descs.append(sent.lower().strip())
    return preprocessed_descs

In [ ]:
train['preprocessed_name'] = preprocess_name(train['name'])
test['preprocessed_name'] = preprocess_name(test['name'])

train['preprocessed_description'] = preprocess_desc(train['item_description'])
test['preprocessed_description'] = preprocess_desc(test['item_description'])

#### - Sau bước trên, có khả năng xuất hiện những dòng có name hoặc item_description rỗng. Ta sẽ loại bỏ những hàng này.

# Loại bỏ hàng có name/description rỗng trong train


In [ ]:
n_rows = train.shape[0]
train = train[train.preprocessed_name != ''].reset_index(drop=True)
train = train[train.preprocessed_description != ''].reset_index(drop=True)
print('Đã xóa {} hàng'.format(n_rows - train.shape[0]))

# **4. Featurization**

# **4.1. Vectorization**

- Với dữ liệu dạng text, để sử dụng làm input trong model dự đoán, văn bản cần được mã hóa dưới dạng số thực hoặc vector. 
- Mỗi loại sản phẩm có thể có từ ngữ thể hiển đặc điểm riêng của sản phẩm đó mà có thể quyết định giá của sản phẩm, còn một số từ ngữ xuất hiện nhiều có thể chưa chắc quyết định đến giá của sản phẩm nên chúng ta nên sử dụng mô hình TF-IDF để xử lý để chống việc các từ ngữ xuất hiện liên tục thường xuyên được đánh trọng số cao, còn những từ ngữ xuất hiện ít, quan trọng thì đánh trọng số thấp.

Ta sử dụng thư viện vector hóa văn bản của sklearn là TfidfVectorizer cho dữ liệu dạng văn bản là name và item_description.
- TF-IDF chuyển văn bản thành một vector mà trong đó, giá trị của từng phần tử đánh giá tầm quan trọng của một từ trong một văn bản. Giá trị đó được tính theo công thức:

\begin{align*}
\mathbf{tf}(t, d) = \frac{ \mathbf{f}(t, d)}{ max \{ \mathbf{f}(w, d) : w ∈ d \} }
\end{align*}<br>


\begin{align*}
\mathbf{idf}(t, D) = \log{ \frac{|D|}{| \{d ∈ D : t ∈ d \} |}}
\end{align*}<br>


\begin{align*}
\mathbf{tfidf}(t, d, D) = \mathbf{tf}(t, d) * \mathbf{idf}(t, D)
\end{align*}<br>

>Trong đó:
>   *       tf(t, d): tần suất xuất hiện của từ t trong văn bản d
>   *       f(t, d): Số lần xuất hiện của từ t trong văn bản d
>   *       max({f(w, d) : w ∈ d}): Số lần xuất hiện của từ có số lần xuất hiện nhiều nhất trong văn bản d
>   *       |D|: Tổng số văn bản trong tập D
>   *       |{d ∈ D : t ∈ d}|: thể hiện số văn bản trong tập D có chứa từ t.*

#### Dùng Tfidf Vectorizer để xử lý name và item_description

In [ ]:
tfidf_name = TfidfVectorizer(ngram_range=(1, 3), min_df=3, max_features=250000)
train_name = tfidf_name.fit_transform(train['preprocessed_name'])
test_name = tfidf_name.transform(test['preprocessed_name'])

tfidf_descp = TfidfVectorizer(ngram_range=(1, 3), min_df=5, max_features=500000)
train_descp = tfidf_descp.fit_transform(train['preprocessed_description'])
test_descp = tfidf_descp.transform(test['preprocessed_description'])

#### Sau khi vector hóa name và item_description:

In [ ]:
train_name.shape

Feature name đã trở thành vector mới với 250000 điểm.

In [ ]:
train_descp.shape

Feadture item_description đã trở thành vector mới với 500000 điểm.

# **4.2. Label Binarize**

Với dữ liệu dạng label, cần biến đổi các thuộc tính này thành một dạng số đại diện hơn để có thể dễ dàng đưa vào input của model. 

Ta sử dụng thư viện của sklearn là LabelBinarizer.

- Label Binarizer gom các lable thành vector, trong đó giá trị của mỗi lable là 0 hoặc 1 thể hiện nó có xuất hiện hay không.

####  Dùng LabelBinarizer để xử lý các dữ liệu lable.

*brand_name*

In [ ]:
lb_brand_name = LabelBinarizer(sparse_output=True)
train_brand = lb_brand_name.fit_transform(train['brand_name'])
test_brand = lb_brand_name.transform(test['brand_name'])

In [ ]:
train_brand.shape

brand_name giờ trở thành vector với 4808 điểm.

*item_condition*

In [ ]:
lb_item_cond_id = LabelBinarizer(sparse_output=True)
train_item_condition_id = lb_item_cond_id.fit_transform(train['item_condition_id'])
test_item_condition_id = lb_item_cond_id.transform(test['item_condition_id'])

In [ ]:
train_item_condition_id.shape

item_condition_id giờ trở thành vector 5 với điểm.

*shipping*

In [ ]:
lb_shipping = LabelBinarizer(sparse_output=True)
train_shipping = lb_shipping.fit_transform(train['shipping'])
test_shipping = lb_shipping.transform(test['shipping'])

In [ ]:
train_shipping.shape

shipping giờ trở thành vector với 1 điểm.

*gencat, subcat1, subcat2*

In [ ]:
lb_gencat = LabelBinarizer(sparse_output=True)
train_gencat = lb_gencat.fit_transform(train['gencat_name'])
test_gencat = lb_gencat.transform(test['gencat_name'])

lb_subcat1 = LabelBinarizer(sparse_output=True)
train_subcat1 = lb_subcat1.fit_transform(train['subcat1_name'])
test_subcat1 = lb_subcat1.transform(test['subcat1_name'])

lb_subcat2 = LabelBinarizer(sparse_output=True)
train_subcat2 = lb_subcat2.fit_transform(train['subcat2_name'])
test_subcat2 = lb_subcat2.transform(test['subcat2_name'])

In [ ]:
train_gencat.shape

train_gencat giờ trở thành vector với 11 điểm.

In [ ]:
train_subcat1.shape

train_subcat1 giờ trở thành vector với 114 điểm.

In [ ]:
train_subcat2.shape

train_subcat2 giờ trở thành vector với 871 điểm.

# **4.3. Gộp các feature sau khi vector hóa thành một ma trận**

Đưa tất cả các dữ liệu đã được xử lý thành một ma trận để có thể đưa vào mô hình train.

Sử dụng hàm hstack của scipy để stack các cột dữ liệu lại với nhau.

Xóa bớt những biến không cần thiết, giải phóng và tránh tràn bộ nhớ.

In [ ]:
new_train = hstack((train_name, train_descp, train_brand, 
                    train_item_condition_id, train_shipping,
                    train_gencat, train_subcat1, train_subcat2)).tocsr()

del train_name, train_descp, train_brand, train_item_condition_id, train_shipping, train_gencat, train_subcat1, train_subcat2

In [ ]:
del test

new_test = hstack((test_name, test_descp, test_brand, 
                   test_item_condition_id, test_shipping, 
                   test_gencat, test_subcat1, test_subcat2)).tocsr()

del test_name, test_descp, test_brand, test_item_condition_id, test_shipping, test_gencat, test_subcat1, test_subcat2

In [ ]:
new_train.shape

Tập train mới bây giờ là một ma trận với 755810 điểm

# **5. Chọn Model**

#### Sử dụng hàm lỗi RMSLE

In [ ]:
# https://www.kaggle.com/carlolepelaars/understanding-the-metric-rmsle
def rmsle(y, y_pred):
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

def evaluate(y, y_pred):
    return rmsle(np.expm1(y), np.expm1(y_pred))

#### Cross validation

Để tránh hiện tượng Overfit, ta sử dụng kỹ thuật cross validation: 
- Trích từ tập training data ra một tập con nhỏ (validata set) và thực hiện việc đánh giá mô hình trên tập con nhỏ này.
- Ta chia tập training ra k tập con không có phần tử chung, có kích thước gần bằng nhau. 
- Tại mỗi lần kiểm thử, một trong số k tập con được lấy ra làm validata set.

Ở đây, ta chọn tách 10% của tập train để test.

In [ ]:
def model_train_predict(model, matrix):
    X = matrix
    X_train, X_test, y_train, y_test = train_test_split(X, train['price'], test_size=0.1)

    model.fit(X_train, y_train)
    y_pre = model.predict(X_test)

    return y_pre, y_test

# **5.1. So sánh giữa các model**
( Do bộ nhớ có hạn, không thể chạy tất cả model trong cùng một notebook; vì vậy ta phải chạy riêng từng model một và lưu lại kết quả. Kết quả ghi lại đã được tổng hợp trong phần 5.2)

- Đầu tiên, ta sử dụng mô hình hồi quy cơ bản là hồi quy tuyến tính, cùng nâng cấp của nó là Ridge. 
- Sau đó sử dụng các mô hình boosting XGBoost và LightGBM. 

> Boosting là một thuật toán Ensemble Learning với phương pháp tổng hợp các weak learner thành một strong learner, trong đó lần lượt học các weak learner sao cho weak learner sau cải thiện weak learner trước.
> Một thuật toán Boosting là Gradient Boosting. Trong Gradient Boosting, các learner sau sẽ học sai số của learner trước nó. Do đó, ensemble model sẽ dần dần khiến cho loss = 0.
> LightGBM và XGBooost là các phiên bản cải tiến của GBM (Gradient Boosting Model), với thuật toán cơn bản là Cây quyết định (Decision Tree), nó thực hiện việc xây dựng tuần tự nhiều Cây quyết định và tiến hành học tập. Điểm khác biệt giữa LightGBm và XGBoost sẽ được đề cập ở phần LightGBM.

# **5.1.1. Linear Model**

Hồi quy tuyến tính (Linear Regression) là một phương pháp phân tích quan hệ giữa biến phụ thuộc Y với một hay nhiều biến độc lập X.

Công thức tổng quát: 

> $y = Xw + e$

Trong đó: 
> * $y$: biến phụ thuộc (biến ta sẽ dự đoán giá trị)
> * $X$: vector biến độc lập(các đặc trưng)
> * $w$: vector hệ số
> * $e$: sai số

Tương tự với tất cả các cặp $(X_{i}, y_{i})$ của N điểm dữ liệu. Điều kiện bài toán trở thành tim tổng sai số là nhỏ nhất, tương đương với việc tìm $w$ để hàm mất mát đạt giá trị nhỏ nhất:

\begin{align*} 
L(w)&= \frac{1}{2} \sum_{i = 1}^{N} \left ( y_{i} - X_{i}w \right ) ^{2}\\\ &= \frac{1}{2} \left \| y - Xw \right \|_{2}^{2} 
\end{align*}

Trong đó: 
* $y$: vector cột chứa tất cả các output 
* $X$: ma trận dữ liệu input

Nghiệm tối ưu của bài toán có dạng: 
\begin{align*} 
w = (X^{T}X)^{\dagger}X^{T} y
\end{align*}
Với 
* $X^{T}$: ma trận chuyển vị của X
* $X^{\dagger}$: ma trận giả nghịch đảo của X

In [ ]:
linear = LinearRegression()

# y_pred_Linear, y_test = model_train_predict(linear, new_train)

# rmsle_Linear = evaluate(y_test, y_pred_Linear)
# print('giá trị RMSLE của LinearRegression: {}'.format(rmsle_Linear))

**Nhận xét:** Sau nhiều giờ chạy, ta thấy Linear Regression chạy tốn quá nhiều thời gian và bộ nhớ.

=> Ta sẽ coi giá trị RMSLE của Linear Regression là inf.

# **5.1.2. Ridge**

Regularization cơ bản là thay đổi mô hình một chút để tránh overfitting trong khi vẫn giữ được tính tổng quát của nó (tính tổng quát là tính mô tả được nhiều dữ liệu, trong cả tập training và test).Ta sẽ tìm cách di chuyển nghiệm của bài toán tối ưu hàm mất mát tới một điểm gần nó. Hướng di chuyển sẽ là hướng làm cho mô hình ít phức tạp hơn mặc dù giá trị của hàm mất mát có tăng lên một chút.

Kỹ thuật regularization phổ biến nhất là thêm vào hàm mất mát một số hạng nữa. Số hạng này thường dùng để đánh giá độ phức tạp của mô hình. Số hạng này càng lớn, thì mô hình càng phức tạp.

Linear Regression với l2 regularization được gọi là Ridge Regression. Trong đó, $\lambda$ là số hạng regularization, thường là một số dương nhỏ để giảm độ phức tạp của mô hình và không làm giảm chất lượng nghiệm so với Linear Regression. 
Hàm mất mát của Ridge có dạng:

\begin{align*} 
L(w) = \frac{1}{2} \left \| y - Xw \right \|_{2}^{2} + \lambda \left \| w \right \|_{2}^{2}
\end{align*} 

In [ ]:
ridge = Ridge(solver='auto', fit_intercept=True, alpha=4.5, max_iter=200, normalize=False, tol=0.01)

# y_pred_Ridge, y_test = model_train_predict(ridge, new_train)

# rmsle_Ridge = evaluate(y_test, y_pred_Ridge)
# print('giá trị RMSLE của Ridge: {}'.format(rmsle_Ridge))

# **5.1.3. XGBoost**

XGBoost tối ưu các tài nguyên tính toán bằng cách xây dựng các cây Decision Tree một cách song song cùng các thuật toán tối ưu khác.

Ưu điểm của XGBoost:

* Tốc độ xử lý

    * XGBoost thực hiện tinh toán song song nên tốc độ xử lý có thể tăng gấp 10 lần so với GBM. Ngoài ra, XGboost còn hỗ trợ tính toán trên Hadoop.
* Overfitting

    * XGBoost áp dụng cơ chế Regularization nên hạn chế đáng kể hiện tượng Overfitting (GBM không có regularization).
* Sự linh hoạt

    * XGboost cho phép người dùng sử dụng hàm tối ưu và chỉ tiêu đánh giá của riêng họ, không hạn chế ở những hàm cung cấp sẵn.
* Xử lý missing value

    * XGBoost bao gồm cơ chế tự động xử lý missing value bên trong nó. Vì thế, có thể bỏ qua bước này khi chuẩn bị dữ liệu cho XGBoost.
* Tự động cắt tỉa

    * Tính năng tree pruning hỗ trợ việc tự động bỏ qua những leaves, nodes không mang giá trị tích cực trong quá trình mở rộng tree.

In [ ]:
xgb_model = XGBRegressor(n_estimators=200, learning_rate = 0.5, max_depth = 20, min_child_weight = 10)

# y_pred_xgb, y_test = model_train_predict(xgb_model, new_train)

# rmsle_xgb = evaluate(y_test, y_pred_xgb)
# print('giá trị RMSLE của XGBoost: {}'.format(rmsle_xgb))

# **5.1.4. LightGBM**

LightGBM phát huy mạnh hiệu suất khi tập dataset có kích thước cực lớn do LightGBM sử dụng cơ chế leaf-wise thay vì level-wise như các thuật toán khác.
- leaf-wise chỉ mở rộng tree theo 1 trong 2 hướng so với cả 2 hướng của level-wise, tức là số lượng tính toán của Light GBM chỉ bằng 1/2 so với XGBoost.

Ưu điểm của LightGBM:
* tốc độ cao
* kích thước lớn
* hỗ trợ học trên GPU

In [ ]:
lgbm_model = LGBMRegressor(n_estimators=200, learning_rate=0.5, num_leaves=125)

# y_pred_Lgbm, y_test = model_train_predict(lgbm_model, new_train)

# rmsle_Lgbm = evaluate(y_test, y_pred_Lgbm)
# print('giá trị RMSLE của LightGBM: {}'.format(rmsle_Lgbm))

**Nhận xét:** Sau khi chạy, ta thấy LightGBM yêu cầu bộ nhớ nhiều hơn khả năng notebook có thể cung cấp.

=> Ta sẽ coi giá trị RMSLE của LightGBM là inf.

# **5.2. Tổng kết quá trình chọn model**
Qua một vài tính toán, ta có giá trị rmsle tương đối của các model
<!DOCTYPE html>
<html>
<head>
<style>
table, th, td {
  border: 1px solid black;
  border-collapse: collapse;
}
th, td {
  padding: 5px;
}
th, td {
  text-align: left;
}
</style>
</head>
<body>

<table style="width:100%">
<caption>RMSLE value</caption>
  <tr>
    <th>Model</th>
    <th>RMSLE</th> 
  </tr>
  <tr>
    <td>Linear</td>
    <td>INF</td>
  </tr>
  <tr>
    <td>Ridge</td>
    <td>0.4415784934872398</td>
  </tr>
  <tr>
    <td>XGBoost</td>
    <td>0.4616445960329693</td>
  </tr>
  <tr>
    <td>LightGBM</td>
    <td>INF</td>
  </tr>
</table>

</body>
</html>

Nhận xét:
* Model tuyến tính Ridge cho kết quả khá tốt, vượt trội so với Linear Regression về tốc độ học, ít chiếm bộ nhớ và tăng độ chính xác đối với tập dữ liệu lớn bới Regularization.
* Boosting cho kết quả không quá cao hơn so với mô hình tuyến tính.

Dựa vào giá trị rmsle, ta chọn model Ridge.

# **6. Dự đoán**

#### Sử dụng model Ridge để train:

In [ ]:
model = Ridge(solver='auto', fit_intercept=True, alpha=4.5, max_iter=200, normalize=False, tol=0.01)

In [ ]:
X_train = new_train
y_train = train['price']

model.fit(X_train, y_train)

In [ ]:
X_test = new_test

#### Dự đoán

In [ ]:
preds = model.predict(X_test)
preds

# **7. Submit**

#### Đọc file submit mẫu:

In [ ]:
submission = pd.read_csv('sample_submission_stg2.csv')
submission

#### Thay price trong submit mẫu bằng price đã dự đoán:

In [ ]:
submission.loc[:, 'price'] = np.expm1(preds)
submission

#### Xuất file submission.csv để submit:

In [ ]:
submission.to_csv('submission.csv', index=False)